In [ ]:
from aqchem.chemistry import Solute, elements
from aqchem.equilibria import Equilibrium, EqSystem, charge_balance, atom_balance
from IPython.display import Latex, display
import periodictable
%matplotlib inline

In [ ]:
def show(s): display(Latex('$'+s+'$'))

In [ ]:
substances = Hp, OHm, NH4p, NH3, H2O, Cupp, CuNH31pp, CuNH32pp, CuNH33pp, CuNH34pp, CuNH35pp, Cu2OH2pp, CuOH3m, CuOH4mm, CuOH2 = [#, CuOHp, CuOH2, =
    Solute(n, latex_name=l, formula=periodictable.formula(n)) for n, l in [
        ('H{+}', 'H^+'), ('HO{-}', 'OH^-'), ('NH3 + H{+}', 'NH_4^+'),
        ('NH3', 'NH_3'), ('H2O', 'H_2O'), ('Cu{2+}', 'Cu^{2+}'), ('Cu{2+}NH3', 'Cu(NH_3)^{2+}'),
        ('Cu{2+}(NH3)2', 'Cu(NH_3)_2^{2+}'), ('Cu{2+}(NH3)3', 'Cu(NH_3)_3^{2+}'),
        ('Cu{2+}(NH3)4', 'Cu(NH_3)_4^{2+}'), ('Cu{2+}(NH3)5', 'Cu(NH_3)_5^{2+}'), 
        ('2Cu{2+} + 2HO{-}', 'Cu_2(OH)_2^{2+}'),
        ('Cu{2+} + 3HO{-}', 'Cu(OH)_3^-'), ('Cu{2+} + 4HO{-}', 'Cu(OH)_4^{2-}'),
        ('Cu{2+} + 2HO{-}', 'Cu(OH_2)(s)'),
    ]]
CuOH2.solid = True

In [ ]:
show(', '.join([s.latex_name for s in substances])) # + r'(\mathrm{%s})' % (str(s.formula.atoms) + ', ' + str(s.charge))

In [ ]:
init_conc = {Hp: 1e-7, OHm: 1e-7, NH4p: 0, NH3: 1.0, Cupp: 1e-2, 
            CuNH31pp: 0, CuNH32pp: 0, CuNH33pp: 0, CuNH34pp: 0, CuNH35pp: 0,
            H2O: 55.5, Cu2OH2pp: 0, CuOH2: 0, CuOH3m: 0, CuOH4mm: 0}

In [ ]:
# Data från Jämviktskonstanttabell, Avdelningen för oorganisk kemi 2007
H2O_c = init_conc[H2O]
w_autop = Equilibrium({H2O: 1}, {Hp: 1, OHm: 1}, 10**-14/H2O_c)
NH4p_pr = Equilibrium({NH4p: 1}, {Hp: 1, NH3: 1}, 10**-9.26)
CuOH2_s = Equilibrium({CuOH2: 1}, {Cupp: 1, OHm: 2}, 10**-18.8)
CuOH_B3 = Equilibrium({CuOH2: 1, OHm: 1}, {CuOH3m: 1}, 10**-3.6)
CuOH_B4 = Equilibrium({CuOH2: 1, OHm: 2}, {CuOH4mm: 1}, 10**-2.7)
Cu2OH2 = Equilibrium({Cupp: 2, H2O: 2}, {Cu2OH2pp: 1, Hp: 2}, 10**-10.6 / H2O_c**2)
CuNH3_B1 = Equilibrium({CuNH31pp: 1}, {Cupp: 1, NH3: 1}, 10**-4.3)
CuNH3_B2 = Equilibrium({CuNH32pp: 1}, {Cupp: 1, NH3: 2}, 10**-7.9)
CuNH3_B3 = Equilibrium({CuNH33pp: 1}, {Cupp: 1, NH3: 3}, 10**-10.8)
CuNH3_B4 = Equilibrium({CuNH34pp: 1}, {Cupp: 1, NH3: 4}, 10**-13.0)
CuNH3_B5 = Equilibrium({CuNH35pp: 1}, {Cupp: 1, NH3: 5}, 10**-12.4)
equilibria = w_autop, NH4p_pr, CuNH3_B1, CuNH3_B2, CuNH3_B3, CuNH3_B4, CuNH3_B5, Cu2OH2, CuOH_B3, CuOH_B4, CuOH2_s
skip_subs, skip_eq = (4, 4) # (0, 0), (1, 1), (3, 3), (4, 4), (11, 9)
simpl_subs = substances[:-skip_subs]
simpl_eq = equilibria[:-skip_eq]
simpl_c0 = {k: init_conc[k] for k in simpl_subs}

In [ ]:
show(', '.join([s.latex_name for s in simpl_subs]))
show('~')
from math import log10
for eq in simpl_eq:
    ltx = eq.latex()
    show(ltx + '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~lgK = {0:12.5g}'.format(log10(eq.params)))
[(idx, s.name, init_conc[s]) for idx, s in enumerate(simpl_subs)]

In [ ]:
eqsys = EqSystem(simpl_eq, simpl_subs)
import sympy as sp
sp.init_printing()
x, i = sp.symarray('x', eqsys.ns), sp.symarray('i', eqsys.ns)
f, piv, cbs = eqsys.f(x, i, norm=True, rref=False)
#subs = [(_, sp.exp(_)) for _ in x]
#[_.subs(subs).powsimp() for _ in f]
f

In [ ]:
sp.Matrix(1, len(f), lambda _, q: f[q]).jacobian(x)

In [ ]:
len(f), eqsys.ns

In [ ]:
#eqsys.root(simpl_c0, method='lm', rref=False, norm=True, pres_norm=True, scaling=1e12, delta=1e3, logC=True)
#eqsys.root({H2O: 55.5, Hp: 1e-10, OHm: 1e-3, NH3: 1.0, NH4p: 1e-3}, delta=1e-6, method='lm')
eqsys.root(simpl_c0, method='lm', logspace=True, x0=[0]*eqsys.ns)

In [ ]:
import numpy as np
#Cout_logC, inits_logC, success = eqsys.plot(simpl_c0, NH3, np.logspace(-3, 1, 20), norm=True, pres_norm=True, rref=False,
#                            scaling=1e12, method='lm', charge=False, logC=True, presw=1000)
Cout_logC, inits_logC, success = eqsys.plot(simpl_c0, NH3, np.logspace(-3, 1, 20), logspace=True, method='lm')#, x0=[0]*eqsys.ns)
all(success)

In [ ]:
import numpy as np
_1, _2, success = eqsys.plot(simpl_c0, NH3, np.logspace(-3, 1, 10), norm=True, pres_norm=True, rref=False,
                            scaling=1e6, init_guess=Cout_logC, init_iter=50, charge=False)
all(success)

In [ ]:
import numpy as np
x, inits_out, success = eqsys.plot(simpl_c0, NH3, np.logspace(-3, 1, 10), delta=1e-4, method='lm',
                                   carry=False, logC=True, pres1st=True, presw=100) #, init_iter=50)#, pres1st=True, norm=True, scaling=1)
all(success)

In [ ]:
eqsys.plot_errors(x, inits_out, NH3)

In [ ]:
for eq in equilibria:
    show(eq.latex())

In [ ]:
reaction_system = EqSystem(equilibria, substances)

In [ ]:
import sympy as sp
#sp.init_printing()
y = sp.symarray('y', len(substances))
s = sp.symbols('s')

ys = []
class LatexSymbol(sp.Symbol):
    def _print_latex(self):
        return self.name
  
for sbs in reaction_system.substances:
    ys.append(LatexSymbol('[' + sbs.latex_name + ']'))
    print(ys[-1].name)
ys

In [ ]:
qk = reaction_system.qk(y, scaling=s, norm=True)
qk

In [ ]:
charge_balance(substances, y)

In [ ]:
atom_balance(substances, y, 1)

In [ ]:
reaction_system.equilibrium_quotients(y)

In [ ]:
chg_vec = reaction_system.charge_balance_vector()
atm_vecs, atm_nrs = reaction_system.atom_balance_vectors()
import numpy as np
arr = np.array(atm_vecs)
arr.shape, atm_nrs, arr, chg_vec

In [ ]:
import sympy
M = sympy.Matrix([chg_vec] + atm_vecs)
rref, pivot = M.rref()
M, rref, pivot

In [ ]:
rref, pivot = reaction_system.rref()
atm_nrs = reaction_system.atom_balance_vectors()[1]
[0] + atm_nrs
#reaction_system.independent_atoms_from_pivot(pivot)

In [ ]:
qk

In [ ]:
c0arr = np.array([init_conc[k] for k in reaction_system.substances])
scaling = 1.0
logC = False
f, j = reaction_system.num_cb_factory(c0arr*scaling, True, scaling=scaling, logC=logC)
c0arr

In [ ]:
x0 = c0arr+1e-17
f(np.log(x0) if logC else x0)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
jmat = j(np.log(x0) if logC else x0)
plt.imshow(jmat, cmap='binary', interpolation='None')
jmat[:7, :4].tolist()

In [ ]:
len(substances)

In [ ]:
from scipy.optimize import root
x0 = [1]*len(substances)
root(f, x0, jac=j)

In [ ]:
reaction_system.root(init_conc, scaling=scaling, logC=logC, delta=1e-15)

In [ ]:
i = sp.symarray('i', len(y))
print(i)
reaction_system.f(y, i, scaling=s, reduced=True)

In [ ]:
f_elim, j_elim, elim, elm_cbs = reaction_system.num_cb_factory(c0arr*scaling, True, scaling=scaling, logC=logC, reduced=True)
print(elim)

In [ ]:
c0arr_elim = np.array([x for idx, x in enumerate(c0arr) if idx not in elim]) + 1e-13
x0elim = np.log(c0arr_elim) if logC else c0arr_elim
f_elim(x0elim)

In [ ]:
jmat = j_elim(x0elim)
plt.imshow(jmat, cmap='binary', interpolation='None')
x0elim

In [ ]:
jmat

In [ ]:
reaction_system.root(init_conc, scaling=1e8, init_iter=100)